# Extracting land surface temprature data using planetary computer

## This notebook highlights the steps taken to extract land surface temperature from MODIS Land Surface Temperature using planetary computer.

### First, install and import all necessary dependencies.


In [21]:
!pip install localtileserver

import leafmap
import planetary_computer
import pystac_client
import xarray as xr
import stackstac
import rioxarray
import rasterio
from pystac_client import Client
from dask_gateway import GatewayCluster


### Create the  Dask Scheduler which will handle computations within this notebook

In [22]:
cluster = GatewayCluster()  # Creates the Dask Scheduler. Might take a minute.

client = cluster.get_client()

cluster.adapt(minimum=4, maximum=24)
print(cluster.dashboard_link)

https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.adf4f49249144e969376a326a9c9ce82/status


### Retrieve the image collection from stac , stating the required filters in this case the bounding box for Southern Africa and the date range

In [23]:
stac = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1/",
    modifier=planetary_computer.sign_inplace,
)
first_image_collection = stac.search(
    collections=['modis-11A2-061'],
    bbox= [19.37253760833874,-30.388133302695742 ,36.68699073333874,-8.685402342342151],
    datetime="2020-01-01/2020-12-31",
)

items = first_image_collection.item_collection()


### Stack  images into a data array which makes it easy for computations

In [24]:
scene_data = (
    stackstac.stack(
        items,
        epsg=3857,
        resolution=1000,  
        assets=["LST_Day_1km"], 
        chunksize=1000,
    )     
)

scene_data

<xarray.DataArray 'stackstac-cec99a61c6d19380f6c5a37c20003484' (time: 1001,
                                                                band: 1,
                                                                y: 4866, x: 4030)>
dask.array<fetch_raster_window, shape=(1001, 1, 4866, 4030), dtype=float64, chunksize=(1, 1, 1000, 1000), chunktype=numpy.ndarray>
Coordinates: (12/22)
  * time                   (time) datetime64[ns] NaT NaT NaT NaT ... NaT NaT NaT
    id                     (time) <U41 'MYD11A2.A2020361.h21v11.061.202106619...
  * band                   (band) <U11 'LST_Day_1km'
  * x                      (x) float64 1.112e+06 1.113e+06 ... 5.141e+06
  * y                      (y) float64 0.0 -1e+03 ... -4.864e+06 -4.865e+06
    start_datetime         (time) <U20 '2020-12-26T00:00:00Z' ... '2020-01-01...
    ...                     ...
    modis:vertical-tile    (time) int64 11 10 9 12 11 10 9 ... 10 9 12 11 10 9
    created                (time) <U20 '2021-03-07T19:03:28Z' ... '2021-01-05...
    eo:cloud_cover         (time) object None 44 43 None 5 ... 66 None 3 52 56
    title                  <U46 '8-day daytime 1km grid Landsurface Temperature'
    raster:bands           object {'unit': 'Kelvin', 'scale': 0.02, 'data_typ...
    epsg                   int64 3857
Attributes:
    spec:        RasterSpec(epsg=3857, bounds=(1112000, -4866000, 5142000, 0)...
    crs:         epsg:3857
    transform:   | 1000.00, 0.00, 1112000.00|\n| 0.00,-1000.00, 0.00|\n| 0.00...
    resolution:  1000

### Calculate the median value of the data array across the “time” dimension.

In [25]:
median_image = scene_data.median(dim="time").compute()


### Convert the computed array to raster to display on Leafmap.

In [26]:
median_image = median_image.rio.set_crs("EPSG:3857", inplace=True)
image = median_image.rio.to_raster('output.tif')
image

In [27]:
m = leafmap.Map()
m.add_raster('output.tif', layer_name='layer1',colormap='hot')
m


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

### Finally  write  raster to a storage blob where we can downlod it for further analyisis.
### This requires you to have your own container in Azure so make sure to replace  The SAS key, container name and link to container with your own. 


In [29]:
import getpass
import azure.storage.blob

sas_token = getpass.getpass()  # prompts for the sas_token
container_client = azure.storage.blob.ContainerClient(
    "https://rejoice.blob.core.windows.net/climatedata",
    container_name="climatedata",
    credential=sas_token,
)

 ········


In [ ]:
import io

with io.BytesIO() as buffer:
    median_image.rio.to_raster(buffer, driver="COG")
    buffer.seek(0)
    blob_client = container_client.get_blob_client("temp2020Africa-wb.tif")
    blob_client.upload_blob(buffer, overwrite=True)


#### These are the steps taken in extracting the land surface temp data for all years. Further analysis is done in Qgis